In [ ]:
import sys
import time
import os

import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tf.keras import optimizers, regularizers
from tf.keras.preprocessing import sequence
from tf.keras.models import Model, Sequential, Sequential, load_model
from tf.keras.layers import Dense, Dropout, Concatenate, Reshape, Activation, Flatten, Input, LSTM
from tf.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint

In [ ]:
'''We imposed that the maximum number of semesters that can be taken by an undergraduate student is 16. 
   Therefore, we created a 15-point GPA time series for each student to predict the last semeseter GPA. 
   Since not all students required 16 semesters to finish their study, we filled in missing values 
   for time points beyond the last semester.
   
   For example, if the last recorded data of a student is in the sixth semester,
   we obtain the GPA series of the student as follows:
   
   gpa_series = [2.9, 1.8, 2.8, 4.0, 3.1]
   
   Notice that We exclude GPA for the sixth semester because it is used for the y-value (predicted last GPA).
   
   We further padded the above list by filling in some random numerical value (e.g., -1 in our case)
   to create a 15-point time series, therefore we have:
   
   gpa_series = [2.9, 1.8, 2.8, 4.0, 3.1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]''' 

#load the processed data
processedData = pd.read_csv('data_example.csv')

#drop rows with LA status
processedData = processedData[(processedData['ProgramStatus']!='LA')] 

#drop redundant columns
drop_columns = ["smt", "leave", "n_gpa", "ProgramStatus"]
processedData = processedData.drop(drop_columns, axis=1)

#encode the following categorical columns
encodedData = pd.get_dummies(processedData, columns=['Campus', 'AcademicProgramDescription'])

#shift column "smt_gpa_last" and "last_gpa" to the very right of the table
lb = ["smt_gpa_last", "last_gpa"]
encodedData = encodedData[[c for c in encodedData if c not in lb] + [c for c in lb if c in encodedData]]

#separate data modality
D = encodedData.values
Ds = X[:, :16] #sequential data
Df = X[:, 16:] #tabular data

#obtain the last semester of each student (excluding the semester for last_gpa)
last_smt = Df[:,-2].astype(np.int64) - 1

#create a list of semesters after semester last_smt
idx_last_smt = [[x for x in range(last_smt[i], 16)] for i in range(len(last_smt))]

#create a list of GPA
gpas=[]
for i in range(len(Ds)):
    gpa = np.array(Ds[i])
    gpa = np.delete(gpa, idx_last_smt[i]).tolist()
    gpas.append(gpa)

#fill in missing values in the list with zeros
gpas = [[0 if x != x else x for x in gpas[i]] for i in range(len(gpas))]

#create padded sequences
padded_Ds = sequence.pad_sequences(gpas, padding='post', 
                                   value=-1.0, 
                                   dtype='float32')

padded_Ds = np.expand_dims(padded_Ds, axis=-1)

#combine together padded_Ds and Df
D_new = [ds + df for ds, df in zip(padded_Ds, Df)]

#create data input
y = D_new[:,-1]
X = D_new[:,:-1]

#split train, valid, test set
test_size = 0.20
seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

#separate the sequential data (s) and tabular data(t)
Xs_train = X_train[:, :16]
Xt_train = X_train[:, 16:]
Xs_test = X_test[:, :16]
Xt_test = X_test[:, 16:]

#Scale and Normalize the data
scaler_train = StandardScaler().fit(Xt_train)
Xt_train = scaler_train.transform(Xt_train)
scaler_test = StandardScaler().fit(Xt_test)
Xt_test = scaler_test.transform(Xt_test)

In [ ]:
'''Define three models: 
   1) ann: MLP with several layers for tabular data input only, 
   2) lstm: LSTM layers for sequential data input only, and 
   3) lstm_ann: LSTM layers and MLP layers are merged by concenating the last layer of each structure.
      The model is for two-headed input.
   4) proposed: The proposed model  described in the paper (kind of)
   The parameters used in the following models (num. of layer, neurons, regularizer, etc) are different 
   with those proposed in the paper because we should have perform hyperparameter optimization first. 
   Here we only gave the general idea about the architecture of such models.'''

def ann():
    t_model = Sequential()
    t_model.add(Dense(64, activation="relu", kernel_initializer='uniform', input_dim=Xt_train.shape[1]))
    t_model.add(Dropout(0.4))
    t_model.add(Dense(256, activation="relu", kernel_initializer='uniform'))
    t_model.add(Dropout(0.4))
    t_model.add(Dense(512, activation="relu", kernel_initializer='uniform'))
    t_model.add(Dropout(0.4))
    t_model.add(Dense(256, activation="relu", kernel_initializer='uniform'))
    #                 kernel_regularizer=regularizers.l1(0.01), bias_regularizer=regularizers.l1(0.01)))
    t_model.add(Dropout(0.5))
    t_model.add(Dense(64, activation="relu", kernel_initializer='uniform'))
    #t_model.add(Dropout(0.4))
    t_model.add(Dense(1, activation='relu'))
    #adam = optimizers.Adam(lr=0.0001)
    t_model.compile(
        loss='mean_squared_error',
        optimizer='adam',
        metrics=['mean_absolute_error'])
    return(t_model)

def lstm():
    inputs = Input(shape=(Xs_train.shape[1], 1))
    #inputs2 = Masking(mask_value=special_value, input_shape=(None,paddedX_train.shape[1],1))(inputs2)
    lstm = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(inputs)
    #lstm = Dropout(0.5)(lstm)
    lstm = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(lstm)
    #lstm = Dropout(0.3)(lstm)
    #lstm = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.05))(lstm)
    #lstm = Dropout(0.5)(lstm)

    model_tot = Dense(64, activation='relu', kernel_initializer='glorot_uniform')(lstm)
    model_tot = Dropout(0.1)(lstm)
    model_tot = Flatten()(model_tot)
    output = Dense(1, activation='relu')(model_tot)
    
    model = Model(inputs=inputs, outputs=output)
    adam = optimizers.Adam(lr=0.00005)
    model.compile(
        loss='mean_squared_error', 
        optimizer=adam, 
        metrics=["mean_absolute_error"])
    return model

def lstm_ann():
    inputs1 = Input(shape=(Xt_train.shape[1],))
    nn = Dense(64, activation='relu', kernel_initializer='glorot_uniform')(inputs1)
    #nn = Dropout(0.4)(nn)
    nn = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(nn)
    #nn = Dropout(0.4)(nn)
    nn = Dense(512, activation='relu', kernel_initializer='glorot_uniform')(nn)
    #nn = Dropout(0.4)(nn)
    nn = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(nn)
    #nn = Dropout(0.4)(nn)
    nn = Dense(64, activation='relu')(nn)
    nn = Reshape((1, 64))(nn)

    inputs2 = Input(shape=(Xs_train.shape[1], 1))
    #inputs2 = Masking(mask_value=special_value, input_shape=(None,paddedX_train.shape[1],1))(inputs2)
    lstm = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(inputs2)
    #lstm = Dropout(0.2)(lstm)
    lstm = LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01))(lstm)
    #lstm = Dropout(0.4)(lstm)
    #lstm = LSTM(64, return_sequences=True)(lstm)
    #lstm = Dropout(0.3)(lstm)

    model_tot = Concatenate([nn, lstm], axis=1)
    model_tot = Dense(64, activation='relu', kernel_regularizer=l2(0.01), kernel_initializer='glorot_uniform')(model_tot)
    #model_tot = Dropout(0.4)(model_tot)
    model_tot = Flatten()(model_tot)
    output = Dense(1, activation='relu')(model_tot)
    
    model = Model(inputs=[inputs1, inputs2], outputs=output)
    adam = optimizers.Adam(lr=0.00005)
    model.compile(
        loss='mean_squared_error', 
        optimizer=adam, 
        metrics=["mean_absolute_error"])
    return model

def proposed():
    inputs1 = Input(shape=(Xt_train.shape[1],))
    inputs2 = Input(shape=(Xs_train.shape[1], 1))
    
    nn = Dense(128, activation='relu', kernel_initializer='glorot_uniform')(inputs1)
    lstm = LSTM(units=64, return_sequences=True)(inputs2)
    
    concat = Concatenate(axis=1)([nn, lstm])
    nn = Dense(64, activation='relu', kernel_initializer='glorot_uniform')(concat)
    lstm = LSTM(units=32, return_sequences=True)(lstm)
    
    concat = Concatenate(axis=1)([concat, nn, lstm])
    nn = Dense(32, activation='relu', kernel_initializer='glorot_uniform')(concat)
    lstm = LSTM(units=16, return_sequences=True)(lstm)
    
    concat = Concatenate(axis=1)([concat, nn, lstm])
    nn = Dense(16, activation='relu', kernel_initializer='glorot_uniform')(concat)
    lstm = LSTM(units=8, return_sequences=True)(lstm)
    
    concat = Concatenate(axis=1)([concat, nn, lstm])
    nn = Dense(8, activation='relu', kernel_initializer='glorot_uniform')(concat)
    lstm = LSTM(units=4, return_sequences=True)(lstm)
    
    model_tot = Concatenate(axis=1)([concat, nn, lstm])

    model_tot = Dense(units=4, activation='relu', kernel_regularizer=regularizers.l2(0.01), 
                      kernel_initializer='glorot_uniform')(model_tot)

    model_tot = Flatten()(model_tot)
    output = Dense(1, activation='relu')(model_tot)
    
    model = Model(inputs=[input_hr, input_st], outputs=output)
    opt = optimizers.Adam(lr=1e-3)
    model.compile(
        loss='mean_squared_error', 
        optimizer=opt, 
        metrics=["mean_absolute_error"])
    return model

In [ ]:
'''Training the model(s) using k-fold cross validation'''

model1 = ann()
model2 = lstm()
model3 = lstm_ann()
model4 = proposed()

#Here we only demonstrate it for the proposed model (model4)
kf = KFold(n_splits=5, shuffle=True, random_state=seed)
mae_cv = []
mse_cv = []
r2_cv = []

for i in range(5):
    result = next(kf.split(Xt_train), None)
    t_train = Xt_train[result[0]]
    s_train = Xs_train[result[0]]
    p_train = y_train[result[0]]
    t_valid = Xt_train[result[1]]
    s_valid = Xs_train[result[1]]
    p_valid = y_train[result[1]]
    
    mcp_save = ModelCheckpoint('model4/weight_cv.hdf5', save_best_only=True, monitor='val_loss', mode='min')
    history = model4.fit([t_train, s_train],
                          p_train, 
                          validation_data=([t_valid, s_valid], p_valid), 
                          epochs=100,
                          batch_size=64, 
                          verbose = 1,
                          callbacks=[EarlyStopping(monitor='val_loss', patience=10, mode='min'), mcp_save])

    pred = model4.predict([t_valid, s_valid])
    pred = pred.reshape(p_valid.shape[0], 1)[:,-1]
    
    mae_cv.append(mean_absolute_error(p_valid, pred))
    mse_cv.append(mean_squared_error(p_valid, pred)) 
    r2_cv.append(r2_score(p_valid, pred))
    
print("Average mse = %.2f (+/- %.2f)" % (np.mean(mse_cv), np.std(mse_cv)))
print("Best mse = %.2f " % (np.min(mse_cv)))

In [ ]:
'''Evaluate the model(s) to the testing dataset'''
